# Interacting with Recommenders, Campaigns and Filters <a class="anchor" id="top"></a>

Now that the Retail Demo Store has trained models for 3 different use cases (Recommended for You, Customers who viewed X also viewed and Ranking), we need to integrate them into our application. Amazon Personalize can make recommendations available via an Application Programming Interface (API). In addition, Amazon Personalize includes features that allow you to easily integrate into applications and provide benefits like real time vending of recommendations based on recent application activity.

In this notebook, you will interact with campaigns and filters in Amazon Personalize.

1. [Introduction](#intro)
1. [Interact with Recommenders](#interact-recommenders)
1. [Interact with Campaigns](#interact-campaigns)
1. [Filters](#filters)
1. [Create Filters](#create-filters)
1. [Using Filters](#using-filters)
1. [Real-time Events](#real-time)
1. [Wrap Up](#wrapup)

To run this notebook, you need to have run the previous notebooks, [`01_Data_Layer.ipynb`](01_Data_Layer.ipynb), and [`02_Training_Layer.ipynb`](02_Training_Layer.ipynb), where you created a dataset and imported interaction, item, and user metadata data into Amazon Personalize, created recommenders, solutions, and campaigns.

## Introduction <a class="anchor" id="intro"></a>
[Back to top](#top)

At this point, you should have 2 Recommenders and one deployed Campaign. Once they are active, there are resources for querying the recommendations, and helper functions to digest the output into something more human-readable. 

In this Notebook we will interact with Recommenders and Campaigns and get recommendations. 

We will create and interact with filters and send live data to Amazon Personalize to see the effect of real-time interactions on recommendations.

The following diagram shows the resources that we will create in this section. The part we are building  in this notebook highlighted in blue with a dashed outline.

![Workflow](images/03_Inference_Layer_Resources.jpg)

To get started, once again, we need to import libraries, load values from previous notebooks, and load the SDK.

In [ ]:
import time
from time import sleep
import json
from datetime import datetime
import uuid
import random
import boto3
import pandas as pd

In [ ]:
#retrieves previously stored variables 
%store -r 

In [ ]:
personalize = boto3.client('personalize')
personalize_runtime = boto3.client('personalize-runtime')

# Establish a connection to Personalize's event streaming
personalize_events = boto3.client(service_name='personalize-events')

## Adding some helper functions to make results more readable

First, let's create a supporting function to help make sense of the results returned by a Personalize recommender or campaign. Personalize returns only an `item_id`, and potentially information about whether that item was part of a promotion filter. This is great for keeping data compact, but it means you need to query a database or lookup table to get a human-readable result for the notebooks. We will create a helper function to return a human-readable result from the synthetic dataset.

<div class="alert alert-block alert-warning">
<b>Note:</b> NOTE: If you want to lookup information as part of the API request, consider implementing Real-Time Personalization APIs - https://github.com/aws-samples/personalization-apis.
</div>



In [ ]:
def get_item_name_from_id ( item_id ):
    item_name = item_metadata_df [item_metadata_df ['id'] == item_id]['name'].values[0]
    return item_name

In [ ]:
def get_item_category_from_id ( item_id ):
    item_name = item_metadata_df [item_metadata_df ['id'] == item_id]['category'].values[0]
    return item_name

In [ ]:
def get_item_style_from_id ( item_id ):
    item_name = item_metadata_df [item_metadata_df ['id'] == item_id]['style'].values[0]
    return item_name

Great! Now we have a way of rendering results. 

## Interact with Recommenders <a class="anchor" id="interact-recommenders"></a>
[Back to top](#top)

Now that the recommenders have been trained, lets have a look at the recommendations we can get for our users!

### "Customers who viewed X also viewed" Recommender

"Customers who viewed X also viewed" requires an item and a user as input, and it will return customers also viewed based on an item that you specify.

The cells below will handle getting recommendations from the "Customers who viewed X also viewed" Recommender and rendering the results. Let's see what the recommendations are for an item.

We will be using the `recommenderArn`, the `itemId`, the `userId`, as well as the number or results we want, `numResults`.

#### Select a User

We'll just pick a random user for simplicity (there are about 6,000 users with user IDs assigned in sequential order). Feel free to change the `user_id` below and execute the following cells with a different user to get a sense for how the recommendations change.

In [ ]:
user_id = 555
user_metadata_df[user_metadata_df['id']==user_id]

We'll just pick a random product for simplicity. Feel free to change the `product_id` below and execute the following cells with a different product to get a sense for how the recommendations change.

In [ ]:
product_id = item_metadata_df.sample(1)["id"].values[0]

print ('productId: ', product_id)

First lets get the response directly from the get_recommendations API, which by default returns 25 items, but can be adjusted.

In [ ]:
get_recommendations_response = personalize_runtime.get_recommendations(
    recommenderArn = workshop_recommender_customers_who_viewed_arn,
    itemId = str(product_id),
    userId = str(user_id),
    numResults = 20
)

In [ ]:
print (get_recommendations_response)

Congrats, this is your first list of recommendations! This list is fine, but it would be better to see the recommendations for products render in a nice dataframe. Again, let's create a helper function to achieve this.

In [ ]:
# Update DF rendering
pd.set_option('display.max_rows', 30)

def get_new_recommendations_df_viewed_x(recommendations_df, item_id, user_id):
    # Get the item name
    original_item_name = get_item_name_from_id(item_id)
    # Get the recommendations
    get_recommendations_response = personalize_runtime.get_recommendations(
        recommenderArn = workshop_recommender_customers_who_viewed_arn,
        itemId = str(item_id),
        userId = str(user_id),
    )
    # Build a new dataframe of recommendations
    item_list = get_recommendations_response['itemList']
    recommendation_list = []
    for item in item_list:
        item_name = get_item_name_from_id(item['itemId'])
        recommendation_list.append(item_name)
    new_rec_df = pd.DataFrame(recommendation_list, columns = [original_item_name])
    # Add this dataframe to the old one
    recommendations_df = pd.concat([recommendations_df, new_rec_df], axis=1)
    return recommendations_df

Now, let's test the helper function with several different items. Let's sample some data from our dataset to test our "Customers who viewed X also viewed" Recommender. Grab 5 random items from our dataframe.

In [ ]:
samples = item_metadata_df.sample(5)
samples

In [ ]:
viewed_x_also_viewed_recommendations_df = pd.DataFrame()
items = samples.id.tolist()

for item in items:
    viewed_x_also_viewed_recommendations_df = get_new_recommendations_df_viewed_x(viewed_x_also_viewed_recommendations_df, item, user_id)

viewed_x_also_viewed_recommendations_df

Based on the random product selected above, do the similar item recommendations from Personalize make sense? Keep in mind that the "Customers who viewed X also viewed" recommendations are based on the interactions we generated as input into the solution creation process above.

You may notice that some items look the same, hopefully not all of them do (this is more likely with a smaller # of interactions, which will be more common with a small dataset).

### "Recommended for you" Recommender

"Recommended for you" supports personalization of the items for a specific user based on their past behavior and can intake real time events in order to alter recommendations for a user without retraining. 

With this use case, Amazon Personalize automatically filters items the user purchased based on the `userId` that you specify and "Purchase" events. For better performance, include "Purchase" events along with the required "View" events. 

In [ ]:
get_recommendations_response = personalize_runtime.get_recommendations(
    recommenderArn = workshop_recommender_recommended_for_you_arn,
    userId = str(user_id),
    numResults = 20
)

In [ ]:
item_list = get_recommendations_response['itemList']

df = pd.DataFrame()
df['Item'] = [ itm['itemId'] for itm in item_list ]
df['Name'] = [ get_item_name_from_id ( itm['itemId']) for itm in item_list ]
df['Category'] = [ get_item_category_from_id ( itm['itemId']) for itm in item_list ]
df['Style'] = [ get_item_style_from_id ( itm['itemId']) for itm in item_list ]
display (df)


Since "Recommended for you" relies on having a sampling of users, let's load the data we need for that and select 3 random users.

In [ ]:
samples = user_metadata_df.sample(3)
samples

Now we render the recommendations for our 3 random users from above. After that, we will explore real-time interactions before moving on to Personalized Ranking.

"Recommended for you" requires only a user as input, and it will return items that are relevant for that particular user. In this particular case the item is a product.

The cells below will handle getting recommendations from the "Recommended for you" Recommender and rendering the results. 

We will be using the `recommenderArn`, the `userId` as well as the number or results we want, `numResults`.

Again, we create a helper function to render the results in a nice dataframe.

#### API call results

In [ ]:
# Update DF rendering
pd.set_option('display.max_rows', 30)

def get_new_recommendations_df_recommended_for_you(recommendations_df, user_id):
    # Get the recommendations
    get_recommendations_response = personalize_runtime.get_recommendations(
        recommenderArn = workshop_recommender_recommended_for_you_arn,
        userId = str(user_id),
    )
    # Build a new dataframe of recommendations
    item_list = get_recommendations_response['itemList']
    recommendation_list = []
    for item in item_list:
        item_name = get_item_name_from_id(item['itemId'])
        recommendation_list.append(item_name)
    new_rec_df = pd.DataFrame(recommendation_list, columns = [user_id])
    # Add this dataframe to the old one
    recommendations_df = pd.concat([recommendations_df, new_rec_df], axis=1)
    return recommendations_df

In [ ]:
recommendations_df_users = pd.DataFrame()
users = samples.id.tolist()

for user in users:
    recommendations_df_users = get_new_recommendations_df_recommended_for_you(recommendations_df_users, user)

recommendations_df_users

## Applying the discount context

We'll get the user recommendations when discount context is applied for comparison. This is using the "contextual metadata" feature of Amazon Personalize. In this case, our context is whether a discount was applied.

In [ ]:
get_recommendations_response = personalize_runtime.get_recommendations(
    recommenderArn = workshop_recommender_recommended_for_you_arn,
    userId = str(user_id),
    numResults = 20,
    context = {'DISCOUNT':'Yes'} # Here we provide the context for the recommendations
)

item_list_context = get_recommendations_response['itemList']

df = pd.DataFrame()
df['Item'] = [ itm['itemId'] for itm in item_list_context ]
df['Name'] = [ get_item_name_from_id ( itm['itemId']) for itm in item_list_context ]
df['Category'] = [ get_item_category_from_id ( itm['itemId']) for itm in item_list_context ]
df['Style'] = [ get_item_style_from_id ( itm['itemId']) for itm in item_list_context ]
display (df)

Let us compare it to the previous set of recommendations without context.

In [ ]:
df = pd.DataFrame()
df['Item (No context)'] = [ itm['itemId'] for itm in item_list ]
df['Name (No context)'] = [ get_item_name_from_id ( itm['itemId']) for itm in item_list ]

df['Item (Context)'] = [ itm['itemId'] for itm in item_list_context ]
df['Name (Context)'] = [ get_item_name_from_id ( itm['itemId']) for itm in item_list_context ]
display (df)

## Interact with Campaigns <a class="anchor" id="interact-campaigns"></a>
[Back to top](#top)


### Personalized Ranking

The core use case for personalized ranking is to take a collection of items and to render them in priority or probable order of interest for a user. For an ECOMMERCE application you want to dynamically render a personalized shelf/rail/carousel based on some information (category, style, season, etc…). This may not be information that you have in your metadata, so an item metadata filter will not work, however you may have this information within your system to generate the item list. You can use this campaign to rank the products listed for each category and the featured products list, as well as ranking catalog search results displayed in a search widget.

To demonstrate this, we will use the same user from before and a random collection of items.

#### Get Featured Products List

First let's get the list of featured products from the Products data.

In [ ]:
featured_products = item_metadata_df[item_metadata_df['featured']==True]

#### Rank Featured Products

Using the featured products list just retrieved, first we'll create a list of item IDs that we want to rank for a specific user. This ranking allows us to provide ranked products based on the user's behavior. These behaviors should be consistent to the same persona that was mentioned above (since we're going to use the same `user_id`).

In [ ]:
unranked_product_ids = list (featured_products['id'])

df = pd.DataFrame()
df['Item'] = [ itm for itm in unranked_product_ids ]
df['Name'] = [ get_item_name_from_id ( itm) for itm in unranked_product_ids ]
df['Category'] = [ get_item_category_from_id ( itm) for itm in unranked_product_ids ]
df['Style'] = [ get_item_style_from_id ( itm) for itm in unranked_product_ids ]
display (df)

Now let's have Personalize rank the featured product IDs based on our random user.

In [ ]:
response = personalize_runtime.get_personalized_ranking(
    campaignArn = workshop_rerank_campaign_arn,
    inputList = unranked_product_ids,
    userId = str(user_id)
)
reranked = response['personalizedRanking']
print(json.dumps(response['personalizedRanking'], indent = 4))

We will add the ranked items as a second column to the original dataframe, for a side-by-side comparison.

In [ ]:
df = pd.DataFrame()
df['Original List'] = [ itm for itm in unranked_product_ids]
df['Original List Name'] = [ get_item_name_from_id ( itm) for itm in unranked_product_ids ]

df['Reranking'] = [ itm['itemId'] for itm in reranked]
df['Reranking Name'] = [ get_item_name_from_id ( itm['itemId']) for itm in reranked ]

display (df)

You can see above how each entry was re-ordered based on the model's understanding of the user. This is a popular task when you have a collection of items to surface a user that cannot be easily categorized in your metadata, for instance "featured products" which are curated by a person.


# Filters <a class="anchor" id="filters"></a>

## Create Filters <a class="anchor" id="create-filters"></a>
[Back to top](#top)

Personalize can utilize either [static or dynamic filters](https://docs.aws.amazon.com/personalize/latest/dg/filter.html). Static filters are where the filter properties are built into the filter itself, which makes invocation simpler, but gives less flexibility. An example of this would be an accessories category filter, which invokes the get_recommendations_response api with the specific filter of CATEGORY_L1 = accessories. In order to create a recommendation for each filter, that would require 10+ filters. Personalize also supports dynamic filters, where the values can be passed at runtime, allowing for a single filter of CATEGORY_L1, where the actual category is passed at runtime. 

Filters can be created for fields of both Items and Events. 

A few common use cases for dynamic filters in ECOMMERCE are:

* Categorical filters based on Item Metadata - Often your item metadata will have information about the item. Filtering on these can provide recommendations within that data.

* Range based filters based on Item Metadata - Personalize supports range operations in both static and dynamic filters. Filtering based on a range can be used to create recommendations such as "Whats on sale now".

* Events - you may want to filter out certain events and provide results based on those events, such as moving a title from a "recommended for you" recommendation to a "purchase again" recommendations.



### Create Category Filter

Since there are a lot of categories to filter on, we will create a dynamic filter using the dynamic variable $CATEGORY, this will allow us to pass in the variable at runtime rather than create a static filter for each category.

In [ ]:
category_filter_name = 'filter-category'

try:
    create_category_filter_response = personalize.create_filter(
        name = category_filter_name,
        datasetGroupArn = workshop_dataset_group_arn,
        filterExpression = 'INCLUDE ItemID WHERE Items.CATEGORY_L1 IN ($CATEGORY)'
    )
    
    category_filter_arn = create_category_filter_response['filterArn']
    print('Creating the Personalize filter with category_filter_arn {}.'.format(category_filter_arn))
    

except personalize.exceptions.ResourceAlreadyExistsException as e:
    category_filter_arn = 'arn:aws:personalize:'+region+':'+account_id+':filter/'+category_filter_name
    print('The Personalize filter {} already exists.'.format(category_filter_name))
    print ('\nWe will be using the existing Personalize Filter with category_filter_arn = {}'.format(category_filter_arn))
    

### Create Purchased Products Filter

Since it's a poor user experience to recommend products that a user has already purchased, we will create a filter that excludes recently purchased products. We'll do this by creating a filter expression that excludes items that have an interaction with an event type of `Purchase` for the user. If you recall from the last notebook, the interactions dataset uses the `Purchase` event type to track purchased items.

In [ ]:
filter_name = 'filter-purchased-products'

try:
    create_purchase_filter_response = personalize.create_filter(
        name=filter_name,
        datasetGroupArn = workshop_dataset_group_arn,
        filterExpression = 'EXCLUDE itemId WHERE INTERACTIONS.event_type in ("Purchase")'
    )
    
    purchase_filter_arn = create_purchase_filter_response['filterArn']

    print('Creating the Personalize filter with purchase_filter_arn {}.'.format(purchase_filter_arn))
    
except personalize.exceptions.ResourceAlreadyExistsException as e:
    purchase_filter_arn = 'arn:aws:personalize:'+region+':'+account_id+':filter/'+filter_name
    print('The Personalize filter {} already exists.'.format(filter_name))
    print ('\nWe will be using the existing Personalize Filter with purchase_filter_arn = {}'.format(purchase_filter_arn))
    

### Wait for Filter Status to Become ACTIVE

The filter should take a minute or so to become active.

In [ ]:
status = None
max_time = time.time() + 60*60 # 1 hours
while time.time() < max_time:
    describe_filter_response = personalize.describe_filter(
        filterArn = purchase_filter_arn
    )
    status = describe_filter_response["filter"]["status"]
    print("Purchase filter: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(15)
    
status = None
max_time = time.time() + 60*60 # 1 hours
while time.time() < max_time:
    describe_filter_response = personalize.describe_filter(
        filterArn = category_filter_arn
    )
    status = describe_filter_response["filter"]["status"]
    print("Category Filter: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(15)

## Using Filters <a class="anchor" id="using-filters"></a>
[Back to top](#top)

Now that the Filters have been created we can use them to filter our recommendations.

## Category Filter<a class="anchor" id="filter">
[Back to top](#top)

To test our purchased products filter, we will request recommendations for a random user with a particular category. 
    
We will create a helper function to return recommendations using a dynamic flilter:

In [ ]:
def get_new_recommendations_df_by_dynamic_filter(recommendations_df, user_id, category_filter_arn, filter_values):
    # Get the recommendations
    get_recommendations_response = personalize_runtime.get_recommendations(
        recommenderArn = workshop_recommender_recommended_for_you_arn,
        userId = str(user_id),
        filterArn = category_filter_arn,
        filterValues = { "CATEGORY": "\"" + filter_values + "\""},
        numResults = 10
    )
    # Build a new dataframe of recommendations
    item_list = get_recommendations_response['itemList']
    recommendation_list = []
    for item in item_list:
        product = get_item_name_from_id(item['itemId'])
        recommendation_list.append(product)
    filter_name = category_filter_arn.split('/')[1]
    new_rec_DF = pd.DataFrame(recommendation_list, columns = [filter_values])
    # Add this dataframe to the old one
    recommendations_df = pd.concat([recommendations_df, new_rec_DF], axis=1)
    return recommendations_df

We will select the first 7 categories for this example.

In [ ]:
unique_category_field_values = list(item_metadata_df['category'].unique()[:7])
unique_category_field_values

In [ ]:
# Iterate through Categories
recommendations_df_category = pd.DataFrame()
for category in unique_category_field_values:
    recommendations_df_category = get_new_recommendations_df_by_dynamic_filter(recommendations_df_category, user_id, category_filter_arn , category)
    
recommendations_df_category

## Real-time Events<a class="anchor" id="event-tracking"></a>
[Back to top](#top)

The next topic is real-time events. Personalize has the ability to listen to events from your application in order to update the recommendations shown to the user. This is especially useful in retail workloads, because what products the user is interested in now may be different than what they were interested in a week ago, a day ago, or even a few minutes ago.

Additionally the events that are recorded via this system are stored until a delete call from you is issued, and they are used as historical data alongside the other interaction data you provided when you train your next models.

Start by creating an event tracker that is attached to the dataset group. This event tracker will add information to the dataset and will influence the recommendations.

### Create Personalize Event Tracker

Let's start by creating an event tracker for our dataset group.

In [ ]:
event_tracker_name = 'personalize-poc-event-tracker'
try: 
    create_event_tracker_response = personalize.create_event_tracker(
        name = event_tracker_name,
        datasetGroupArn=workshop_dataset_group_arn
        )
    event_tracker_arn = create_event_tracker_response['eventTrackerArn']
    print(json.dumps(create_event_tracker_response, indent=2))
    print ('\nCreating the event_tracker with event_tracker_arn = {}'.format(event_tracker_arn))
    tracking_id = create_event_tracker_response['trackingId']
    print ('\nAnd trackingId = {}'.format(tracking_id))
    

except personalize.exceptions.ResourceAlreadyExistsException as e:
    event_tracker_list = personalize.list_event_trackers( 
        datasetGroupArn= workshop_dataset_group_arn
    )['eventTrackers']
    
    event_tracker_arn = event_tracker_list[0]['eventTrackerArn']
    
    describe_event_tracker_response = personalize.describe_event_tracker(
        eventTrackerArn=event_tracker_arn
    )
    tracking_id = describe_event_tracker_response['eventTracker']['trackingId']
    
    print ('\nThe the Event Tracker with event_tracker_name {} already exists'.format(event_tracker_name))
    print ('\nWe will be using the existing Event Tracker with event_tracker_arn = {}'.format(event_tracker_arn))
    print ('\nAnd tracking_id = {}'.format(tracking_id))
    

### Wait for Event Tracker Status to Become ACTIVE

The event tracker should take a minute or so to become active.

In [ ]:
status = None
max_time = time.time() + 60*60 # 1 hours
while time.time() < max_time:
    describe_event_tracker_response = personalize.describe_event_tracker(
        eventTrackerArn = event_tracker_arn
    )
    status = describe_event_tracker_response["eventTracker"]["status"]
    print("EventTracker: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(15)

### Cold User Recommendations

One of the key features of Personalize is being able to cold start users. Cold users are typically those who are new to your site or application and cold starting a user is getting from no personalization to making personalized recommendations in real-time. 

Personalize accomplishes cold starting users via the Event Tracker. Since new users are typically anonymous for a period of time before they create an account or may choose to transact as a guest, personalization is a valuable tool to help convert those anonymous users to transacting users. 

The challenge here is that Personalize needs a `userId` for anonymous users before it can make personalized recommendations. This challenge can be solved by creating a provisional user ID the moment an anonymous user first hits the site. This provisional user ID is then used when streaming events to the Event Tracker and when retrieving recommendations from the Recommendations service. This allows applications to start serving personalized recommendations after the first couple events are streamed to Personalize. Before recommendations can be personalized, Personalize will provide recommendations for popular items as a fallback.

There are some challenges with this approach, though. First is the question of what to do with the provisional user ID when the user creates an account. To maintain continuity of the user's interaction history, client applications can pass the provisional user ID to the user management system when creating a new user account. Another challenge is how to handle a user that anonymously browses the site using multiple devices such as on the mobile device and then on a desktop/laptop. In this case, separate provisional user IDs are generated for sessions on each device. However, once the user creates an account on one device and then signs in with that account on the other device, both devices will starting using the same user ID going forward. A side effect here is that the interaction history from one of the devices will be orphaned. This is an acceptable tradeoff given the benefit of cold starting users earlier and is functionally the same UX without this scheme. Additional logic could be added to merge the interaction history from both prior anonymous sessions when the user creates an account. Also, customer data platforms can be used to help manage this for you.

### Using the Purchased Products Filter

To use purchased products filter, we will request ranked recommendations for a random user. Then we will send a `Purchase` event for one of the recommended products to Personalize using the event tracker created above. Finally, we will request recommendations again for the same user but this time specify our filter. The purchased product would be excluded from the new recommendations.

In [ ]:
# Pick a user ID in the range of test users and fetch 5 recommendations.
user_id = 456
display(user_metadata_df[user_metadata_df['id']==user_id])

In [ ]:
get_recommendations_response = personalize_runtime.get_personalized_ranking(
    campaignArn = workshop_rerank_campaign_arn,
    inputList = unranked_product_ids,
    userId = str(user_id)
)

item_list = get_recommendations_response['personalizedRanking']
df = pd.DataFrame()
df['Item'] = [ itm['itemId'] for itm in item_list ]
df['Name'] = [ get_item_name_from_id ( itm['itemId']) for itm in item_list ]
df['Category'] = [ get_item_category_from_id ( itm['itemId']) for itm in item_list ]
df['Style'] = [ get_item_style_from_id ( itm['itemId']) for itm in item_list ]
display (df)

Next let's randomly select an item from the returned list of recommendations to be our product to purchase.

In [ ]:
product_id_to_purchase = random.choice(item_list)['itemId']
print(f'Product to simulate purchasing: {product_id_to_purchase}')
print(f'Product name: {get_item_name_from_id ( product_id_to_purchase)}')

Next, let's send a `Purchase` event to Personalize to simulate that the product was just purchased.
This will match the criteria for our filter.
In the Retail Demo Store web application, this event is sent for each product in the order after the order is completed.

In [ ]:
response = personalize_events.put_events(
    trackingId = tracking_id,
    userId = str(user_id),
    sessionId = str(uuid.uuid4()),
    eventList = [
        {
            'eventId': str(uuid.uuid4()),
            'eventType': 'Purchase',
            'itemId': str(product_id_to_purchase),
            'sentAt': int(time.time()),
            'properties': '{"discount": "No"}'
        }
    ]
)

# Wait for Purchase event to become consistent.
time.sleep(10)

print(json.dumps(response, indent=2))

Finally, let's retrieve recommendations for the user again, but this time specifying the filter to exclude recently
purchased items. We do this by passing the filter's ARN via the `filterArn` parameter.

In [ ]:
get_recommendations_response = personalize_runtime.get_personalized_ranking(
    campaignArn = workshop_rerank_campaign_arn,
    inputList = unranked_product_ids,
    userId = str(user_id),
    filterArn = purchase_filter_arn
)

item_list = get_recommendations_response['personalizedRanking']
df = pd.DataFrame()
df['Item'] = [ itm['itemId'] for itm in item_list ]
df['Name'] = [ get_item_name_from_id ( itm['itemId']) for itm in item_list ]
df['Category'] = [ get_item_category_from_id ( itm['itemId']) for itm in item_list ]
df['Style'] = [ get_item_style_from_id ( itm['itemId']) for itm in item_list ]
display (df)

The following code will raise an assertion error if the product we just purchased is still recommended.

In [ ]:
found_item = next((item for item in item_list if item['itemId'] == product_id_to_purchase), None)
if found_item:
    assert found_item == False, 'Purchased item found unexpectedly in recommendations'
else:
    print('Purchased item filtered from recommendations for user!')

In [ ]:
%store workshop_dataset_group_arn
%store region
%store role_name
%store s3_access_policy_arn

# Variables required for Optional Notebook 1 - Batch Inference
%store users
%store unranked_product_ids
%store bucket_name
%store workshop_rerank_solution_version_arn
%store item_metadata_df

## Wrap up <a class="anchor" id="wrapup"></a>
[Back to top](#top)

With that you now have a fully working collection of models to tackle various recommendation and personalization scenarios, as well as the skills to manipulate customer data to better integrate with the service, and a knowledge of how to do all this over APIs and by leveraging open source data science tools.

You'll want to make sure that you clean up all of the resources deployed during this POC. We have provided a separate notebook which shows you how to identify and delete the resources in [`04_Clean_Up.ipynb`](04_Clean_Up.ipynb).

You can choose to head to any of the optional notebooks in this folder to continue experimenting with Amazon Personalize.

<div class="alert alert-block alert-warning">
<b>Note:</b> NOTE: Run optional notebooks BEFORE you run the clean-up notebook.
</div>
